In [1]:
import requests
from bs4 import BeautifulSoup as bs4

In [2]:
resp = requests.get(
    url = "https://sfbay.craigslist.org/sfc/apa")

In [3]:
resp.url

'https://sfbay.craigslist.org/search/sfc/apa'

In [4]:
html = bs4(resp.text, "html.parser")

In [5]:
print(html.prettify()[:100])

﻿
<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   SF bay area apartments / housing rentals


In [110]:
apts = html.findAll("p", attrs = {"class":"row"})

In [111]:
len(apts)

100

In [112]:
one_apt = apts[1]

In [183]:
apt_info = one_apt.find("span", attrs={"class":"housing"})

In [184]:
apt_info

<span class="housing">
                2br -
                850ft<sup>2</sup> -
            </span>

In [185]:
apt_info.text

'\n                2br -\n                850ft2 -\n            '

In [187]:
apt_info.text.replace("\n", "").replace("-","").split()

['2br', '850ft2']

In [84]:
one_apt.find("time")

<time datetime="2016-09-07 12:20" title="Wed 07 Sep 12:20:07 PM">Sep  7</time>

In [85]:
one_apt.find("time")["datetime"]

'2016-09-07 12:20'

In [86]:
one_apt.find("a", attrs={"class":"hdrlnk"})["data-id"]

'5770748709'

Maybe we should do the string stuff at the pandas level?

In [113]:
dates = [one_apt.find("time")["datetime"] for one_apt in apts]

In [115]:
ids = [one_apt.find("a", attrs={"class":"hdrlnk"})["data-id"] for one_apt in apts]

In [139]:
if apts[5].find("span", attrs={"class":"housing"}):
    print("dasd")

In [144]:
bdrm = apts[10].find("span", attrs={"class":"housing"})

In [150]:
if bdrm:
    info = bdrm.text.replace("\n", "").replace("-","").split()
else:
    info = None

In [173]:
import pandas as pd

In [265]:
from craigslist import CraigslistHousing

cl_h = CraigslistHousing(site='sfbay', area='sfc', category='apa'
                        )
data = pd.DataFrame(cl_h.get_results(limit=30))

In [268]:
data.head(10)

,bedrooms,datetime,geotag,has_image,has_map,id,name,price,url,where
0,1br 1498ft2 ...,2016-09-07 20:10,None,True,True,5771441590,1 large private bedroom & bath in 1500 sq foot...,$2650,http://sfbay.craigslist.org/sfc/apa/5771441590...,potrero hill
1,2br,2016-09-07 20:09,None,True,True,5759868522,Executive -View Condo,$7500,http://sfbay.craigslist.org/sfc/apa/5759868522...,pacific heights
2,1br,2016-09-07 20:08,None,False,False,5771427733,TELEGRAPH HILL GRAND VIEW!,$2850,http://sfbay.craigslist.org/sfc/apa/5771427733...,north beach / telegraph hill
3,2br,2016-09-07 20:07,None,True,True,5771439450,Inner Richmond(Turk/Arguello)Updated 2BR Unit ...,$3375,http://sfbay.craigslist.org/sfc/apa/5771439450...,inner richmond
4,1br,2016-09-07 20:06,None,True,True,5771438221,"New, Clean, Luxury; Walk to AT&T Park, Caltrai...",$1790,http://sfbay.craigslist.org/sfc/apa/5771438221...,SOMA / south beach
5,None,2016-09-07 20:05,None,True,True,5771437480,Cozy apartment used as a 2 bedroom,$3150,http://sfbay.craigslist.org/sfc/apa/5771437480...,russian hill
6,1br 614ft2 ...,2016-09-07 20:04,None,True,True,5771436121,Gorgeous 1-Bedroom Classic SF Apartment in Gre...,$3090,http://sfbay.craigslist.org/sfc/apa/5771436121...,pacific heights
7,2br 1403ft2 ...,2016-09-07 20:03,None,True,True,5771425269,PROPERTY FORCE - Gorgeous 2br/2.5ba SOMA Apart...,$4795,http://sfbay.craigslist.org/sfc/apa/5771425269...,SOMA / south beach
8,2br 1581ft2 ...,2016-09-07 20:02,None,True,True,5764368519,PENTHOUSE - ONE HAWTHORNE - SPECTACULAR VIEWS ...,$5990,http://sfbay.craigslist.org/sfc/apa/5764368519...,SOMA / south beach
9,3br,2016-09-07 20:02,None,True,True,5771434524,Single-Family Home w/ Amazing Views in Miralom...,$4400,http://sfbay.craigslist.org/sfc/apa/5771434524...,glen park


In [337]:
string  = data.bedrooms[5]

In [339]:
#[re.findall("\dbr", string) for string in data.bedrooms]

In [386]:
bednum = []
for string in data.bedrooms:
    if string:
        num = re.findall(r"\dbr", string)
        num = int(num[0].replace("br", ""))
        bednum.append(num)
    else:
        bednum.append(None)

In [391]:
area = []
for string in data.bedrooms:
    if string:
        num = re.findall(r"\d{1,6}ft2", string)
        num = int(num[0].replace("ft2", "")) if num else None
        area.append(num) 
    else:
        area.append(None)